I'd like to look at the gene expression results in a basic way, perhaps look for outliers
or things that don't look like iPSCs. Then I'll filter genes to come up with a set
I want to use for identifying eQTLs.

In [34]:
import os
import subprocess

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns

import ciepy as cpy
import projectpy as ppy

%matplotlib inline

In [42]:
array_meta = pd.read_table(os.path.join(cpy.root, 'data', 'array_metadata.tsv'), index_col=0)
wgs_meta = pd.read_table(os.path.join(cpy.root, 'data', 'wgs_metadata.tsv'), index_col=0, 
                         squeeze=True)
rna_meta = pd.read_table(os.path.join(cpy.root, 'data', 'rna_seq_metadata.tsv'), index_col=0)

vsd = pd.read_table(os.path.join(cpy.root, 'output', 'rna_seq_processing', 
                                 'vst_counts.tsv'), index_col=0)

# Normality of Expression Values

I want to see how normal the expression values are.

In [ ]:
res = stats.mstats.normaltest(vsd, axis=1)

In [ ]:
pd.Series(-1 * np.log(res[1])).hist(bins=20)
plt.ylabel('Number of genes')
plt.xlabel('$-\log$ $p$-value for test of normality');

In [ ]:
(res[1] < 0.05).sum()

It seems like a fair number of genes deviate from normality. However, when I've looked at some
genes graphically, the distributions often aren't too weird. I'm not sure whether I need to 
normalize the data (quantile, map to normal, etc.) or not.